In [1]:
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
heloc = pd.read_excel('heloc_featureset_final.xlsx')
display(heloc.head(n=5))

,RiskPerformance,MSinceOldestTradeOpen,AverageMInFile,NetFractionRevolvingBurden,NetFractionInstallBurden,NumBank2NatlTradesWHighUtilization,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,MSinceMostRecentInqexcl7days
0,Bad,144,84,33,-8,1,20,3,0
1,Bad,58,41,0,-8,-8,2,4,0
2,Bad,66,24,53,66,1,9,0,0
3,Bad,169,73,72,83,3,28,1,0
4,Bad,333,132,51,89,0,12,0,0


In [3]:
riskperf_raw = heloc['RiskPerformance']
features = heloc.drop('RiskPerformance' , axis = 1)
dic={'Good':0, 'Bad':1}
riskperf = riskperf_raw.map(dic)
display(riskperf.head())
display(features.head())

0    1
1    1
2    1
3    1
4    1
Name: RiskPerformance, dtype: int64

,MSinceOldestTradeOpen,AverageMInFile,NetFractionRevolvingBurden,NetFractionInstallBurden,NumBank2NatlTradesWHighUtilization,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,MSinceMostRecentInqexcl7days
0,144,84,33,-8,1,20,3,0
1,58,41,0,-8,-8,2,4,0
2,66,24,53,66,1,9,0,0
3,169,73,72,83,3,28,1,0
4,333,132,51,89,0,12,0,0


In [20]:
from gmean_score import Gscore
from sklearn.metrics import make_scorer
def scre(y_true, y_pred):
    scre = Gscore(y_true, y_pred)
    return scre.g_mean()



#scr = Gscore(y_true, y_pred)

clfs_params = {'LogisticRegression' : dict(clf__C=np.arange(1e-05, 1.0, 0.1)),
            'DecisionTreeClassifier': {'max_depth':[1,2,3,4,5,6,7,8,9,10]},
            'GradientBoosting': {'max_depth':[1,2,3,4,5,6,7,8,9,10]}}
clfs = {'LogisticRegression' : LogisticRegression(),
            'DecisionTreeClassifier': DecisionTreeClassifier(max_depth= 6),
            'GradientBoosting': GradientBoostingClassifier(max_depth= 6)
            }
print(clfs_params)
for clf, clf_name in zip(clfs.values(), clfs.keys()):
    print(clfs_params[clf_name])
    


{'GradientBoosting': {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, 'LogisticRegression': {'clf__C': array([  1.00000000e-05,   1.00010000e-01,   2.00010000e-01,
         3.00010000e-01,   4.00010000e-01,   5.00010000e-01,
         6.00010000e-01,   7.00010000e-01,   8.00010000e-01,
         9.00010000e-01])}, 'DecisionTreeClassifier': {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}}
{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
{'clf__C': array([  1.00000000e-05,   1.00010000e-01,   2.00010000e-01,
         3.00010000e-01,   4.00010000e-01,   5.00010000e-01,
         6.00010000e-01,   7.00010000e-01,   8.00010000e-01,
         9.00010000e-01])}
{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}


y_true = [0,0,0, 1, 1, 0,0,0,0, 1]
y_pred = [1,1,1, 1, 1, 0,0,0,0,0]

scr = Gscore(y_true, y_pred)

print("TP {},FP {},TN {},FN {}".format(*scr.pred()))
print(scr.TP)
print(scr.FP)
print(scr.TN)
print(scr.FN)
print(scr.sensi())
#"Wholesale customers dataset has {} samples with {} features each.".format(*data.shape)

## Models

### Function to run models

In [121]:
# https://www.kaggle.com/vincepota/predicting-customers-who-will-charge-off
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

#max_depth= 6, n_estimators=100, max_features = 0.3

def run_models(X_train, y_train, X_test, y_test):
    
    clfs = {'LogisticRegression' : LogisticRegression() }
    #,           'DecisionTreeClassifier': DecisionTreeClassifier(max_depth= 6),
    #        'GradientBoosting': GradientBoostingClassifier(max_depth= 6)
            
    
    cols = ['Model','True Positive', 'False Positive', 'True Negative', 'False Negative', 'Sensitivity', 'Specificity', 'G_Mean']
    models_report = pd.DataFrame(columns = cols)

    for clf, clf_name in zip(clfs.values(), clfs.keys()):
        clf.fit(X_train, y_train)
        y_train_pred= clf.predict(X_train)
        y_pred = clf.predict(X_test)
        #y_score = clf.predict_proba(X_test)[:,1]
        #print("Type of test",type(y_test))
        #print("Type of pred",type(y_pred))
        #print(y_pred)
        scr = Gscore(y_test.values, y_pred)
        print('computing {} '.format(clf_name))

        tmp = pd.Series({'Model': clf_name,
                         'True Positive' : scr.TP,
                         'False Positive' : scr.FP,
                         'True Negative' : scr.TN,
                         'False Negative' : scr.FN,
                         'Sensitivity': round(scr.sensi(),3),
                         'Specificity': round(scr.speci(),3),
                         'G_Mean': round(scr.g_mean(),3)})

        models_report = models_report.append(tmp, ignore_index = True)
    
    return models_report, y_train_pred


In [122]:
X_train, X_test, y_train, y_test = train_test_split(features, riskperf, test_size=0.2, random_state=0)
models, y_train_pred= run_models(X_train, y_train, X_test, y_test)


computing LogisticRegression 


In [123]:
print(y_pred[0])
print(len(y_train))
print(len(y_train_pred))

0
7892
7892


In [133]:
df = pd.DataFrame(columns=['Index','Train','Pred'])

for i in range(len(y_train)):
    df.loc[i] = [y_train.index[i], y_train.iloc[i] ,y_train_pred[i]]
display(df.head(n=30))

,Index,Train,Pred
0,8497.0,1.0,0.0
1,4197.0,0.0,1.0
2,6864.0,0.0,0.0
3,283.0,1.0,1.0
4,1013.0,1.0,1.0
5,2337.0,0.0,1.0
6,1388.0,0.0,0.0
7,274.0,1.0,0.0
8,4382.0,0.0,1.0
9,3376.0,0.0,0.0


In [132]:

df1 = pd.DataFrame(data=y_train.to_frame())
df1['pred'] = y_train_pred.tolist()
display(df1.head(n=30))

 	Index 	Train 	Pred
0 	8497.0 	1.0 	0.0
1 	4197.0 	0.0 	1.0
2 	6864.0 	0.0 	0.0
3 	283.0 	1.0 	1.0
4 	1013.0 	1.0 	1.0
5 	2337.0 	0.0 	1.0

,RiskPerformance,pred
8497,1,0
4197,0,1
6864,0,0
283,1,1
1013,1,1
2337,0,1
1388,0,0
274,1,0
4382,0,1
3376,0,0


In [94]:
display(y_pred)
display(type(y_pred))
y_train['newcol'] = y_pred.tolist()
display(y_train)
                

array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

numpy.ndarray

8497                                                      1
4197                                                      0
6864                                                      0
283                                                       1
1013                                                      1
2337                                                      0
1388                                                      0
274                                                       1
4382                                                      0
3376                                                      0
484                                                       1
6622                                                      0
9238                                                      0
5601                                                      0
5886                                                      1
162                                                       1
9474                                    

In [134]:
training = X_train.join(df1)
display(X_train.head())
display(y_train.head())
display(training.head(n=30))

,MSinceOldestTradeOpen,AverageMInFile,NetFractionRevolvingBurden,NetFractionInstallBurden,NumBank2NatlTradesWHighUtilization,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,MSinceMostRecentInqexcl7days
8497,172,82,3,-8,0,14,0,0
4197,175,64,53,99,0,34,0,4
6864,311,118,27,-8,0,15,0,0
283,220,69,95,59,2,27,0,0
1013,249,119,58,-8,0,4,5,-8


8497    1
4197    0
6864    0
283     1
1013    1
Name: RiskPerformance, dtype: int64

,MSinceOldestTradeOpen,AverageMInFile,NetFractionRevolvingBurden,NetFractionInstallBurden,NumBank2NatlTradesWHighUtilization,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,MSinceMostRecentInqexcl7days,RiskPerformance,pred
8497,172,82,3,-8,0,14,0,0,1,0
4197,175,64,53,99,0,34,0,4,0,1
6864,311,118,27,-8,0,15,0,0,0,0
283,220,69,95,59,2,27,0,0,1,1
1013,249,119,58,-8,0,4,5,-8,1,1
2337,243,42,91,57,5,31,0,-7,0,1
1388,170,67,4,79,0,34,0,0,0,0
274,375,183,24,-8,0,21,1,0,1,0
4382,90,41,26,83,0,3,0,0,0,1
3376,188,92,0,58,1,25,0,0,0,0


In [78]:
# https://www.kaggle.com/vincepota/predicting-customers-who-will-charge-off
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
#max_depth= 6, n_estimators=100, max_features = 0.3

def optimise_models(X_train, y_train): #, X_test, y_test
    i=0
    clfs = {'LogisticRegression' : LogisticRegression(),
            'DecisionTreeClassifier': DecisionTreeClassifier(),
            'GradientBoosting': GradientBoostingClassifier()
            }
    clfs_params = {'LogisticRegression' : {'C':np.arange(1e-05, 1.0, 0.5)} ,
            'DecisionTreeClassifier': {'max_depth':[1,2,3], 'min_samples_leaf':[50,100,200]},
            'GradientBoosting': {'max_depth':[1,2,3], 'min_samples_leaf':[50,100,200]}
            }
    
    best_models = {}
    
    cols = ['Model','True Positive', 'False Positive', 'True Negative', 'False Negative', 'Sensitivity', 'Specificity', 'G_Mean']
    models_report = pd.DataFrame(columns = cols)
    
    
    #Defining the CV method: Using the Repeated Stratified K Fold
    rskfold = StratifiedShuffleSplit(n_splits=10, test_size=0.4, random_state=2)

    for clf, clf_name in zip(clfs.values(), clfs.keys()):
        print('='*20)
        print('\ncomputing {} '.format(clf_name))
        clf = GridSearchCV(estimator=clf, cv=rskfold, scoring= make_scorer(scre), return_train_score=True,
              param_grid= clfs_params[clf_name]) #, refit='Accuracy'
        
        print('Fitting...')
        start = time() # Get start time
        clf.fit(X_train, y_train.values)
        end = time() # Get end time
        # Success
        #print("{} trained on {} samples.".format(clf.__class__.__name__, len(X_train)))
        print("Training time: ", end-start)
        #results = clf.cv_results_
        print("Best: %f using %s" % (clf.best_score_, clf.best_params_))
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        params = clf.cv_results_['params']
        for mean, stdev, param in zip(means, stds, params):
            print("%f (%f) with: %r" % (mean, stdev, param))
        print("best estimator: " + str(clf.best_estimator_))
        print("best params: " + str(clf.best_params_))
        print('best training score:', round(clf.best_score_,3))
        best_models[clf_name] = clf.best_estimator_
        '''
        #use optimised model to predict
        print('\nPredicting...')
        start = time() # Get start time
        y_pred = clf.best_estimator_.predict(X_test)
        end = time() # Get end time
        print("Prediction time: ", end-start)
        scr = Gscore(y_test.values, y_pred)
        print('Testing score:', round(scr.g_mean(),3))

        tmp = pd.Series({'Model': clf_name,
                         'True Positive' : scr.TP,
                         'False Positive' : scr.FP,
                         'True Negative' : scr.TN,
                         'False Negative' : scr.FN,
                         'Sensitivity': round(scr.sensi(),3),
                         'Specificity': round(scr.speci(),3),
                         'G_Mean': round(scr.g_mean(),3)})

        models_report = models_report.append(tmp, ignore_index = True)
    
    return models_report, best_models
    '''
    return best_models


In [80]:
def models_predict(clfs, X_train, y_train): 
    
    cols = ['Model','True Positive', 'False Positive', 'True Negative', 'False Negative', 'Sensitivity', 'Specificity', 'G_Mean']
    models_report = pd.DataFrame(columns = cols)
  
    
    #Defining the CV method: Using the Repeated Stratified K Fold
    rskfold = StratifiedShuffleSplit(n_splits=10, test_size=0.4, random_state=2)

    for clf, clf_name in zip(clfs.values(), clfs.keys()):
        print('='*20)
        print('\ncomputing {} '.format(clf_name))
        #use optimised model to predict
        print('\nPredicting...')
        start = time() # Get start time
        y_pred = clf.predict(X_train)
        end = time() # Get end time
        print("Prediction time: ", end-start)
        scr = Gscore(y_train.values, y_pred)
        results = cross_val_score(clf, X_train, y_train.values, cv=kfold, scoring=make_scorer(scre))
        print(results)
        print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

        tmp = pd.Series({'Model': clf_name,
                         'True Positive' : scr.TP,
                         'False Positive' : scr.FP,
                         'True Negative' : scr.TN,
                         'False Negative' : scr.FN,
                         'Sensitivity': round(scr.sensi(),3),
                         'Specificity': round(scr.speci(),3),
                         'G_Mean': round(scr.g_mean(),3)})

        models_report = models_report.append(tmp, ignore_index = True)
    
    return models_report

###  Data 

In [79]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
#from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler


# Unscaled
X_train, X_test, y_train, y_test = train_test_split(features, riskperf, test_size=0.2, random_state=0)
models= optimise_models(X_train, y_train) #, X_test, y_test
#print('\nUnscaled data')
#display(models_report)
#print(models)

'''
# Produce the value for 'max_depth'
print("Parameter 'max_depth' is {} for the optimal model.".format(reg.get_params()['max_depth']))
'''



computing GradientBoosting 
Fitting...
Training time:  21.856229782104492
Best: 0.506686 using {'min_samples_leaf': 100, 'max_depth': 2}
0.501244 (0.008729) with: {'min_samples_leaf': 50, 'max_depth': 1}
0.501554 (0.008617) with: {'min_samples_leaf': 100, 'max_depth': 1}
0.501791 (0.008683) with: {'min_samples_leaf': 200, 'max_depth': 1}
0.504806 (0.007751) with: {'min_samples_leaf': 50, 'max_depth': 2}
0.506686 (0.006642) with: {'min_samples_leaf': 100, 'max_depth': 2}
0.505925 (0.007588) with: {'min_samples_leaf': 200, 'max_depth': 2}
0.503137 (0.006356) with: {'min_samples_leaf': 50, 'max_depth': 3}
0.504301 (0.007643) with: {'min_samples_leaf': 100, 'max_depth': 3}
0.506387 (0.006286) with: {'min_samples_leaf': 200, 'max_depth': 3}
best estimator: GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=2,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_sp

'\n# Produce the value for \'max_depth\'\nprint("Parameter \'max_depth\' is {} for the optimal model.".format(reg.get_params()[\'max_depth\']))\n'

In [81]:
from sklearn.model_selection import cross_val_score ,RepeatedStratifiedKFold
models_report = models_predict(models, X_train, y_train)
display(models_report)


computing GradientBoosting 

Predicting...
Prediction time:  0.014991998672485352
[ 0.51441611  0.53942342  0.5091422   0.48868228  0.5129598   0.51692214
  0.51387956  0.50241768  0.50934886  0.51113831  0.50327873  0.53448364
  0.50839681  0.49593521  0.51984797  0.51108598  0.51576712  0.49652208
  0.50155707  0.54266572  0.53533184  0.50843216  0.51081931  0.50885021
  0.50731173]
10-fold cross validation average accuracy: 0.513

computing LogisticRegression 

Predicting...
Prediction time:  0.0
[ 0.48792446  0.50914862  0.48347143  0.46397504  0.47162597  0.4794425
  0.48969796  0.47960315  0.4804401   0.47758326  0.47928829  0.49386183
  0.49450601  0.46397504  0.48740578  0.48639192  0.49176383  0.48060074
  0.45849633  0.50574631  0.4816401   0.48671     0.48304734  0.48697722
  0.47800039]
10-fold cross validation average accuracy: 0.483

computing DecisionTreeClassifier 

Predicting...
Prediction time:  0.0
[ 0.48011399  0.46719186  0.45815569  0.41113756  0.44658893  0.4499

,Model,True Positive,False Positive,True Negative,False Negative,Sensitivity,Specificity,G_Mean
0,GradientBoosting,2666.0,1025.0,3064.0,1137.0,0.701,0.749,0.525
1,LogisticRegression,2652.0,1250.0,2839.0,1151.0,0.697,0.694,0.484
2,DecisionTreeClassifier,2464.0,1096.0,2993.0,1339.0,0.648,0.732,0.474


In [ ]:
def print_coefs(coefs, training_columns):
    sorted_idx = np.argsort(-abs(coefs)) # negative for descending
    for (name, coef) in zip(np.array(training_columns)[sorted_idx], coefs[sorted_idx]):
        print("%s: %f" % (name, coef))

def plot_coefs(coefs, training_columns, title_suffix=''):
    sorted_idx = np.argsort(abs(coefs))
    pos = np.arange(len(coefs)) + .5
    plt.subplot(1, 2, 2)
    plt.barh(pos, coefs[sorted_idx], align='center')
    plt.yticks(pos, np.array(training_columns)[sorted_idx])
    plt.xlabel('Coef value')
    plt.title('Coefficents' + title_suffix)
    plt.xlim(-5.5, 5.5)
    plt.show()


In [ ]:
from  sklearn.ensemblesklearn.e  import GradientBoostingRegressor

scores = cross_val_score(GradientBoostingRegressor(), X, Y, cv=kfold, scoring='neg_mean_squared_error')
print('GBT')
print(scores.mean())

clf = GradientBoostingRegressor()
clf.fit(X, Y)

feature_importance = clf.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())

sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, X.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('GBT Variable Importance')
plt.show()